In [2]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Path to dataset
train_directory = r'C:\Users\chand\Desktop\Projects\Ml models\crop health analysis\PlantVillage\train'

# Ensure the directory exists
if not os.path.exists(train_directory):
    raise FileNotFoundError(f"The directory {train_directory} does not exist.")

# Ensure there are subdirectories in the train directory
if not any(os.path.isdir(os.path.join(train_directory, subdir)) for subdir in os.listdir(train_directory)):
    raise ValueError(f"The directory {train_directory} does not contain any subdirectories representing classes.")

# Data preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

history = model.fit(
    train_generator,
    epochs=6,
    validation_data=validation_generator,
    callbacks=[early_stopping, model_checkpoint]
)

# Save the trained model
model.save('crop_health_model.h5')


Found 1032 images belonging to 2 classes.
Found 257 images belonging to 2 classes.
Epoch 1/6
33/33 [==============================] - 39s 1s/step - loss: 0.7658 - accuracy: 0.7616 - val_loss: 0.3598 - val_accuracy: 0.7743
Epoch 2/6
33/33 [==============================] - 31s 872ms/step - loss: 0.3230 - accuracy: 0.8314 - val_loss: 0.4522 - val_accuracy: 0.8249
Epoch 3/6
33/33 [==============================] - 18s 540ms/step - loss: 0.3562 - accuracy: 0.8372 - val_loss: 0.3227 - val_accuracy: 0.8405
Epoch 4/6
33/33 [==============================] - 18s 534ms/step - loss: 0.2360 - accuracy: 0.9079 - val_loss: 0.2052 - val_accuracy: 0.9416
Epoch 5/6
33/33 [==============================] - 18s 536ms/step - loss: 0.1462 - accuracy: 0.9603 - val_loss: 0.2388 - val_accuracy: 0.9300
Epoch 6/6
33/33 [==============================] - 19s 595ms/step - loss: 0.1603 - accuracy: 0.9448 - val_loss: 0.2253 - val_accuracy: 0.9105
